In [11]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
input_file = 'recommendation_dataset.csv'

data = pd.read_csv(input_file, sep=';', skiprows=0)
data = data.replace(99, np.nan, inplace=True)

In [ ]:
# Convertir todas las columnas numéricas
data.iloc[:, 1:] = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

In [ ]:
data.shape

(73421, 101)

In [ ]:
# Iterar sobre cada columna (restaurante) para predecir los valores faltantes
for column in data.columns[1:]:  # Ignorar la primera columna de nombres de usuarios
    # Separar filas con datos conocidos y faltantes
    known_data = data[data[column].notna()]
    missing_data = data[data[column].isna()]

    # Continuar solo si hay datos faltantes
    if not missing_data.empty:
        # Definir características y etiquetas
        X_train = known_data.drop(columns=[column, 'DATASET'])
        y_train = known_data[column]
        X_missing = missing_data.drop(columns=[column, 'DATASET'])

        # Entrenar el modelo de XGBoost
        model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
        model.fit(X_train, y_train)

        # Predecir los valores faltantes
        predictions = model.predict(X_missing)

        data.loc[data[column].isna(), column] = predictions


Datos imputados:
  DATASET  Restaurant1  Restaurant2  Restaurant3  Restaurant4  Restaurant5  \
0   User1    -7.820000      8.79000    -9.660000    -8.160000        -7.52   
1   User2     3.949656     -0.29000     6.360000     4.370000        -2.38   
2   User3     3.137874      4.65853     4.757423     2.801868         9.03   
3   User4     2.165003      8.35000     2.705641     2.931299         1.80   
4   User5     8.500000      4.61000    -4.170000    -5.390000         1.36   

   Restaurant6  Restaurant7  Restaurant8  Restaurant9  ...  Restaurant91  \
0        -8.50        -9.85         4.17    -8.980000  ...      2.820000   
1        -9.66        -0.73        -5.34     8.880000  ...      2.820000   
2         9.27         9.03         9.27     1.869700  ...      6.675718   
3         8.16        -2.82         6.21    -0.008669  ...      0.018581   
4         1.60         7.04         4.61    -0.440000  ...      5.190000   

   Restaurant92  Restaurant93  Restaurant94  Restaurant95

In [15]:
data.drop(columns=['DATASET'], inplace=True)


In [ ]:
data.to_csv('dataset_imputados.csv', index=False, sep=',')

In [ ]:
def cargar_base_csv(ruta_base):
    """Carga las valoraciones desde el archivo base.csv."""
    with open(ruta_base, 'r') as archivo_base:
        lector = csv.reader(archivo_base)
        return [list(map(float, fila)) for fila in lector]

def poblar_valoraciones(ruta_base, ruta_valoraciones, ruta_salida):
    """Llena el archivo valoraciones.csv con las valoraciones de base.csv."""
    # Cargar las valoraciones desde base.csv
    valoraciones_base = cargar_base_csv(ruta_base)

    # Leer el archivo valoraciones.csv y procesar las líneas
    with open(ruta_valoraciones, 'r') as archivo_val:
        lector = csv.reader(archivo_val, delimiter=';')
        filas_actualizadas = []
        
        for fila in lector:
            usuario = int(fila[0].replace('User', '')) - 1  # Índice del usuario
            restaurante = int(fila[1].replace('Restaurant', '')) - 1  # Índice del restaurante
            valoracion = valoraciones_base[usuario][restaurante]  # Obtener la valoración
            fila_actualizada = f"{fila[0]};{fila[1]};{valoracion}"
            filas_actualizadas.append(fila_actualizada)

    # Guardar las filas actualizadas en un archivo de salida
    with open(ruta_salida, 'w') as archivo_salida:
        for fila in filas_actualizadas:
            archivo_salida.write(fila + '\\n')

# Ejecución de la función
ruta_base = '../assets/data/dataset_imputados.csv'
ruta_valoraciones = '../assets/data/valoraciones.csv'
ruta_salida = '../assets/data/valoraciones_completadasXGBoost.csv'

poblar_valoraciones(ruta_base, ruta_valoraciones, ruta_salida)
print("Archivo valoraciones_completadas.csv generado exitosamente.")